In [128]:
import pandas as pd
import os
import numpy as np

#### DEFINE
output_name = "0MATERIAL_ATTR"


In [129]:
with open("B2D.txt", "r", encoding="utf-8") as file:
    ecc_text = file.read()
with open("B4P.txt", "r", encoding="utf-8") as file:
    hana_text = file.read()

print(os.listdir())

['0AA_ACCDET_TEXT.xlsx', '0AA_RECSTAT_TEXT.xlsx', '0ASSET_AFAB_ATTR.xlsx', '0ASSET_ATTR_TEXT.xlsx', '0BPARTNER_ATTR.xlsx', '0BPARTNER_TEXT.xlsx', '0BP_GRPTYPE_TEXT.png', '0BP_GRP_ATTR.xlsx', '0BP_GRP_HIER.xlsx', '0BP_GRP_TEXT.xlsx', '0CUSTOMER_ATTR.xlsx', '0PLANT_ATTR.xlsx', '2LIS_13_VDITM.xlsx', 'B2D.txt', 'B2D.xlsx', 'B2D_0MATERIAL_ATTR.txt', 'B4D.txt', 'B4D.xlsx', 'B4P.txt', 'B4P_0MATERIAL_ATTR.txt', 'CHECKCCHEC KCCCHEKCHEKCHEKCHE3KCHEEE0C=BHHCHECK.xlsx', 'DefaultOutputName.xlsx', 'fucking_output.xlsx', 'ignore', 'SAPTableCompare.ipynb', 'SAPTableCompare.py', 'test.xlsx', 'Test2.xlsx', 'tt.txt', '~$B4D.xlsx']


In [130]:
def export_df(text):
        ls = [i.strip() for i in text.split("\n") 
                if 
                        ("Append structure " not in i) 
                        and ("Include structure ") not in i 
                        and (len(i) > 1)
                ]
        ls_ls = [i.split(" ") for i in ls]

        #####Define List
        set1 = set([i[-2] for i in ls_ls]) # set2 =  set([i.split(" ")[-3] for i in ls])
        ls1 = [i for i in list(set1) if any(j.isalpha() for j in i)]
        ls1.extend(["QUAN", "DEC"])
        type_ls = list(set(ls1)) 

        ls_name = [ i[0] for i in ls_ls]

        ls_type = []
        for i in ls_ls:
                if i[-2] in type_ls:
                        ls_type.append(f'{i[-2]} {i[-1]}')
                else:
                        ls_type.append(f'{i[-3]} {i[-2]},{i[-1]}')


        ls_combine = []
        for i in range(0,len(ls_ls)):
                ls_combine.append(f'{i} ' + ls_name[i]+" "+ls_type[i])
        ls_combine_ls = [i.split(" ") for i in ls_combine]
        
        return ls_combine_ls

In [75]:
#### Read files and apply def
hana = export_df(hana_text)
ecc = export_df(ecc_text)

# Prepare Dataframe
df_ecc = pd.DataFrame(data=ecc,columns=["e_index","e_name","e_type","e_lgt"])
df_hana = pd.DataFrame(data=hana,columns=["h_index","h_name","h_type","h_lgt"])


In [76]:
#### Check Field Mismatch
ecc_name = set([i[1] for i in ecc])
hana_name = set([i[1] for i in hana])

ecc_u = []
hana_u = []
if ecc_name == hana_name:
    print("Both DataFrames have the same columns.")
else:
    for i in ecc_name - hana_name:
        ecc_u.append(i)
    for i in hana_name - ecc_name:
        hana_u.append(i)

# Make same lenght
max_len = max(len(ecc_u), len(hana_u))
ecc_u += [""] * (max_len - len(ecc_u))  # Pad with empty string
hana_u += [""] * (max_len - len(hana_u))

# Prepare Dataframe
df_mismatched_fields = pd.DataFrame({"Only in HANA": hana_u,"Only in ECC": ecc_u})

# print(df_mismatched_fields.to_string(index=False))  # Properly formatted DataFrame output

In [77]:
#### Check Type Mismatch
ecc_dict = {row[1]: row for row in ecc}  # {C1: [...], C3: [...]}
hana_dict = {row[1]: row for row in hana}  # {C1: [...], C3: [...]}
matching_keys = set(ecc_dict.keys()) & set(hana_dict.keys())

data = []
for i in matching_keys:
    e_row = ecc_dict[i]
    h_row = hana_dict[i]
    if (e_row[3] != h_row[3]) or (e_row[2] != h_row[2]):
        data.append({
            "e_name": e_row[1], "e_type": e_row[2], "e_lgt": e_row[3],
            "trans": ">>>",
            "h_name": h_row[1], "h_type": h_row[2], "h_lgt": h_row[3]
        })

# Convert to DataFrame
df_mismatched_types = pd.DataFrame(data)

# print(df.to_string(index=False))


In [81]:
# Define Excel file path
excel_path = f"{output_name}.xlsx"

# Write to Excel with multiple sheets
with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
    df_ecc.to_excel(writer, sheet_name="ecc", index=False)
    df_hana.to_excel(writer, sheet_name="hana", index=False)
    df_mismatched_types.to_excel(writer, sheet_name="mismatched_types", index=False)
    df_mismatched_fields.to_excel(writer, sheet_name="mismatched_fields", index=False)


print(f"Excel file '{excel_path}'.")


Excel file '0MATERIAL_ATTR.xlsx'.


#The below is the Drafted version for SE15 export .xlsx, however, I found out later that the export version was missing some fields, so...... let's go back and use the first version shall we???

In [163]:
import pandas as pd
import numpy as np

# Read Excel file and skip the first 6 rows
df_hana = pd.read_excel("B4D.xlsx", skiprows=2)
# Read Excel file and skip the first 6 rows
df_ecc = pd.read_excel("B2D.xlsx", skiprows=2)

def export_df(df):
    # Rename
    c1 = df.columns[-1]
    df_rename = df.rename(columns={c1: "decimal"})

    # Select
    columns_to_select = ["Structure Field", "Data Type", "OutLeng", "decimal","Length"]
    df_selected = df_rename[columns_to_select].copy()  # Ensure a copy to avoid warnings

    # Filter for test
    # df_selected = df_selected[df_selected["decimal"].notna()]

    # Transformation
    df_selected = df_selected.fillna("").astype(str)
    df_selected["OutLeng"] = df_selected["OutLeng"].astype(str).str.replace(r"\.0$", "", regex=True)
    df_selected["decimal"] = df_selected["decimal"].astype(str).str.replace(r"\.0$", "", regex=True)
    df_selected["Length"] = df_selected["Length"].astype(str).str.replace(r"\.0$", "", regex=True)
    df_selected["digit"] = np.where(df_selected["OutLeng"].str.len() < 1, df_selected["Length"], df_selected["OutLeng"])
    df_selected.insert(0, "Index", range(1, len(df_selected) + 1)) # Add index
    df_selected["text_row"] = df_selected["Structure Field"] + " " + df_selected["Data Type"] + " " + df_selected["Length"] + "," + df_selected["decimal"]
    df_selected = df_selected.fillna("").astype(str)
    df_selected["text_row"] = df_selected["text_row"].str.split().str.join(" ") #Splits the text into a list of words, automatically removing extra spaces.
    df_selected = df_selected[df_selected["Index"].astype(int) > 2]
    df_selected = df_selected[["text_row"]]
    df_selected.insert(0, "Index", range(1, len(df_selected) + 1))
    df_selected["text_row"] = df_selected["Index"].astype(str) + " " + df_selected["text_row"]



    # Prepare output
    t_text = "\n".join(df_selected["text_row"])
    text_ls = t_text.split("\n")
    ls = []
    for i in text_ls:
        if i[-1] == ",":
            ls.append(i.replace(",",""))
        else:
            ls.append(i)
    
    ls = [i.split(" ") for i in ls]
    return ls

############### Read files and apply def ###############
hana = export_df(df_hana)
ecc = export_df(df_ecc)

df_hana = pd.DataFrame(data=hana)

with pd.ExcelWriter("tttt.xlsx", engine="xlsxwriter") as writer:
    df_hana.to_excel(writer, sheet_name="ttt", index=False)




PermissionError: [Errno 13] Permission denied: 'tttt.xlsx'